## Explore Overfitting and Underfitting

We'll explore two common regularization techniques—weight regularization and dropout—and

In [74]:
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dropout

In [75]:
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load  dataset

dataset = pd.read_csv('/home/francovm/Projects/SSE/data/processed/input_data.csv', sep='\t', encoding='utf-8' ,index_col=0)


X_data = dataset.drop(columns=['Events'])

#one-hot encode target column
Y_data = to_categorical(dataset.Events)


#get number of columns in training data
n_cols = X_data.shape[1]


In [76]:
# split into 67% for train and 33% for test
X_train, X_test, y_train, y_test = train_test_split(X_data, Y_data, test_size=0.33, random_state=seed)

# Baseline Model

In [77]:
baseline_model = Sequential()

#add layers to model
baseline_model.add(Dense(16, activation='relu', input_shape=(n_cols,)))
baseline_model.add(Dense(16, activation='relu'))
baseline_model.add(Dense(2, activation='sigmoid'))

baseline_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'binary_accuracy'])

baseline_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_67 (Dense)             (None, 16)                112       
_________________________________________________________________
dense_68 (Dense)             (None, 16)                272       
_________________________________________________________________
dense_69 (Dense)             (None, 2)                 34        
Total params: 418
Trainable params: 418
Non-trainable params: 0
_________________________________________________________________


In [78]:

#set early stopping monitor so the model stops training when it won't improve anymore
early_stopping_monitor = EarlyStopping(patience=5)

# Fit the model

baseline_history = baseline_model.fit(X_train,y_train, 
                      epochs=20, 
                      validation_split=0.3,
                      batch_size=64,
                      callbacks=[early_stopping_monitor],
                      verbose=1)

Train on 106799 samples, validate on 45771 samples
Epoch 1/20
106799/106799 [==============================] - 5s 43us/step - loss: 0.2092 - acc: 0.9558 - binary_accuracy: 0.9551 - val_loss: 0.1668 - val_acc: 0.9585 - val_binary_accuracy: 0.9583
Epoch 2/20
106799/106799 [==============================] - 3s 30us/step - loss: 0.1764 - acc: 0.9558 - binary_accuracy: 0.9546 - val_loss: 0.1628 - val_acc: 0.9585 - val_binary_accuracy: 0.9569
Epoch 3/20
106799/106799 [==============================] - 3s 31us/step - loss: 0.1736 - acc: 0.9558 - binary_accuracy: 0.9536 - val_loss: 0.1645 - val_acc: 0.9585 - val_binary_accuracy: 0.9559
Epoch 4/20
106799/106799 [==============================] - 3s 31us/step - loss: 0.1721 - acc: 0.9559 - binary_accuracy: 0.9524 - val_loss: 0.1606 - val_acc: 0.9586 - val_binary_accuracy: 0.9551
Epoch 5/20
106799/106799 [==============================] - 3s 31us/step - loss: 0.1705 - acc: 0.9559 - binary_accuracy: 0.9504 - val_loss: 0.1600 - val_acc: 0.9586 - va

In [79]:
smaller_model = Sequential()

#add layers to model
smaller_model.add(Dense(4, activation='relu', input_shape=(n_cols,)))
smaller_model.add(Dense(4, activation='relu'))
smaller_model.add(Dense(2, activation='sigmoid'))

smaller_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'binary_accuracy'])

smaller_model.summary()



_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_70 (Dense)             (None, 4)                 28        
_________________________________________________________________
dense_71 (Dense)             (None, 4)                 20        
_________________________________________________________________
dense_72 (Dense)             (None, 2)                 10        
Total params: 58
Trainable params: 58
Non-trainable params: 0
_________________________________________________________________


In [80]:
# Fit the model

smaller_history = smaller_model.fit(X_train,y_train, 
                      epochs=20, 
                      validation_split=0.3,
                      batch_size=64,
                      callbacks=[early_stopping_monitor],
                      verbose=1)

Train on 106799 samples, validate on 45771 samples
Epoch 1/20
106799/106799 [==============================] - 4s 41us/step - loss: 0.3172 - acc: 0.8232 - binary_accuracy: 0.8766 - val_loss: 0.1675 - val_acc: 0.9585 - val_binary_accuracy: 0.9585
Epoch 2/20
106799/106799 [==============================] - 3s 30us/step - loss: 0.1767 - acc: 0.9558 - binary_accuracy: 0.9556 - val_loss: 0.1664 - val_acc: 0.9585 - val_binary_accuracy: 0.9582
Epoch 3/20
106799/106799 [==============================] - 3s 30us/step - loss: 0.1757 - acc: 0.9558 - binary_accuracy: 0.9556 - val_loss: 0.1652 - val_acc: 0.9585 - val_binary_accuracy: 0.9584
Epoch 4/20
106799/106799 [==============================] - 3s 31us/step - loss: 0.1754 - acc: 0.9558 - binary_accuracy: 0.9557 - val_loss: 0.1703 - val_acc: 0.9585 - val_binary_accuracy: 0.9570
Epoch 5/20
106799/106799 [==============================] - 3s 31us/step - loss: 0.1749 - acc: 0.9558 - binary_accuracy: 0.9557 - val_loss: 0.1670 - val_acc: 0.9585 - va

In [81]:

bigger_model = Sequential()

#add layers to model
bigger_model.add(Dense(512, activation='relu', input_shape=(n_cols,)))
bigger_model.add(Dense(512,activation='relu'))
bigger_model.add(Dense(2, activation='sigmoid'))

bigger_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'binary_accuracy'])

bigger_model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_73 (Dense)             (None, 512)               3584      
_________________________________________________________________
dense_74 (Dense)             (None, 512)               262656    
_________________________________________________________________
dense_75 (Dense)             (None, 2)                 1026      
Total params: 267,266
Trainable params: 267,266
Non-trainable params: 0
_________________________________________________________________


In [82]:
# Fit the model

bigger_history = bigger_model.fit(X_train,y_train, 
                      epochs=20, 
                      validation_split=0.3,
                      batch_size=64,
                      callbacks=[early_stopping_monitor],
                      verbose=1)

Train on 106799 samples, validate on 45771 samples
Epoch 1/20
106799/106799 [==============================] - 10s 93us/step - loss: 0.2393 - acc: 0.9544 - binary_accuracy: 0.9298 - val_loss: 0.1666 - val_acc: 0.9585 - val_binary_accuracy: 0.9585
Epoch 2/20
106799/106799 [==============================] - 9s 81us/step - loss: 0.1753 - acc: 0.9558 - binary_accuracy: 0.9558 - val_loss: 0.1699 - val_acc: 0.9585 - val_binary_accuracy: 0.9585
Epoch 3/20
106799/106799 [==============================] - 9s 81us/step - loss: 0.1720 - acc: 0.9558 - binary_accuracy: 0.9558 - val_loss: 0.1620 - val_acc: 0.9585 - val_binary_accuracy: 0.9585
Epoch 4/20
106799/106799 [==============================] - 9s 84us/step - loss: 0.1707 - acc: 0.9558 - binary_accuracy: 0.9558 - val_loss: 0.1625 - val_acc: 0.9585 - val_binary_accuracy: 0.9585
Epoch 5/20
106799/106799 [==============================] - 9s 85us/step - loss: 0.1696 - acc: 0.9558 - binary_accuracy: 0.9558 - val_loss: 0.1585 - val_acc: 0.9585 - v

In [83]:
def plot_history(histories, key='binary_crossentropy'):
  plt.figure(figsize=(16,10))

  for name, history in histories:
    val = plt.plot(history.epoch, history.history['val_'+key],
                   '--', label=name.title()+' Val')
    plt.plot(history.epoch, history.history[key], color=val[0].get_color(),
             label=name.title()+' Train')

  plt.xlabel('Epochs')
  plt.ylabel(key.replace('_',' ').title())
  plt.legend()

  plt.xlim([-2,max(history.epoch)])


plot_history([('baseline', baseline_history),
              ('smaller', smaller_history),
              ('bigger', bigger_history)])

KeyError: 'val_binary_crossentropy'

<Figure size 1152x720 with 0 Axes>

# Regularization

## *L2*

In [ ]:
l2_model = Sequential()

#add layers to model
l2_model.add(Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(0.001),input_shape=(n_cols,)))
l2_model.add(Dense(512,activation='relu',kernel_regularizer=keras.regularizers.l2(0.001)))
l2_model.add(Dense(2, activation='sigmoid'))

l2_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'binary_accuracy'])

l2_model.summary()

In [ ]:

# Fit the model

l2_model_history =  l2_model.fit(X_train,y_train, 
                      epochs=20, 
                      validation_split=0.3,
                      batch_size=64,
                      callbacks=[early_stopping_monitor],
                      verbose=1)

In [ ]:
plot_history([('baseline', bigger_history),
              ('l2', l2_model_history)])

## *Dropout*

In [ ]:
dpt_model = Sequential()

#add layers to model
dpt_model.add(Dense(512, activation='relu',input_shape=(n_cols,)))
dpt_model.add(Dropout(0.5))
dpt_model.add(Dense(512,activation='relu'))
dpt_model.add(Dropout(0.5))
dpt_model.add(Dense(2, activation='sigmoid'))

dpt_model.compile(optimizer='adam',
                       loss='categorical_crossentropy',
                       metrics=['accuracy', 'binary_accuracy'])

dpt_model.summary()


# Fit the model

dpt_model_history =  dpt_model.fit(X_train,y_train, 
                      epochs=20, 
                      validation_split=0.3,
                      batch_size=64,
                      callbacks=[early_stopping_monitor],
                      verbose=1)

In [ ]:
plot_history([('baseline', bigger_history),
              ('dropout', dpt_model_history)])

In [ ]:
# evaluate the model
scores = bigger_model.evaluate(X_test,y_test)
print((scores[1]*100))

# USE THIS FUNCTION TO FIND BATCH SIZE, JUST EXACTLY BEFORE STARTING YOUR TRAINING/ TESTING.

In [ ]:
def FindBatchSize(model):
    """#model: model architecture, that is yet to be trained"""
    import os, sys, psutil, gc, tensorflow, keras
    import numpy as np
    from keras import backend as K
    BatchFound= 16

    try:
        total_params= int(model.count_params());    GCPU= "CPU"
        #find whether gpu is available
        try:
            if K.tensorflow_backend._get_available_gpus()== []:
                GCPU= "CPU";    #CPU and Cuda9GPU
            else:
                GCPU= "GPU"
        except:
            from tensorflow.python.client import device_lib;    #Cuda8GPU
            def get_available_gpus():
                local_device_protos= device_lib.list_local_devices()
                return [x.name for x in local_device_protos if x.device_type == 'GPU']
            if "gpu" not in str(get_available_gpus()).lower():
                GCPU= "CPU"
            else:
                GCPU= "GPU"

        #decide batch size on the basis of GPU availability and model complexity
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <1000000):
            BatchFound= 64    
        if (os.cpu_count() <16) and (total_params <500000):
            BatchFound= 64  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params <2000000) and (total_params >=1000000):
            BatchFound= 32      
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=2000000) and (total_params <10000000):
            BatchFound= 16  
        if (GCPU== "GPU") and (os.cpu_count() >15) and (total_params >=10000000):
            BatchFound= 8       
        if (os.cpu_count() <16) and (total_params >5000000):
            BatchFound= 8    
        if total_params >100000000:
            BatchFound= 1

    except:
        pass
    try:

        #find percentage of memory used
        memoryused= psutil.virtual_memory()
        memoryused= float(str(memoryused).replace(" ", "").split("percent=")[1].split(",")[0])
        if memoryused >75.0:
            BatchFound= 8
        if memoryused >85.0:
            BatchFound= 4
        if memoryused >90.0:
            BatchFound= 2
        if total_params >100000000:
            BatchFound= 1
        print("Batch Size:  "+ str(BatchFound));    gc.collect()
    except:
        pass

    memoryused= [];    total_params= [];    GCPU= "";
    del memoryused, total_params, GCPU;    gc.collect()
    return BatchFound



#####################################################################################################
#####################################################################################################

In [ ]:
FindBatchSize(baseline_model)